# Combustion Calculator

source: https://support.thunderheadeng.com/tutorials/pyrosim/combustion-calculator/

In [1]:
import sys
import re
import numpy as np

$C_x H_y N_v Cl_w + v_{O_2} O_2 \xrightarrow[]{} v_{CO_2} CO_2 + v_{H_2O} H_2O + v_{CO} CO + v_S \mathrm{Soot} + v_{HCN} HCN + v_{N_2} N_2 + v_{HCl} HCl$

## values

In [38]:
fuel_name = 'PVC'
fuel_formula = "C2H3Cl"

# PVC: C2H3Cl, Y_CO =0.063, Y_S = 0.172

# components of products and Soot 
inYCO_    = 0.343 # input_Y_CO
inYHCN_   = 0.000 # Hyd Cyanide 
inYS_     = 0.237 # soot yield
inXH_     = 0.0   # hydrogen atomic fraction in soot

In [40]:
# air composition
inO2_  = 0.2075588 # 0.2075588 # Volume fraction
inN2_  = 0.7825727 # 0.7825727 # Volume fraction
inH2O_ = 0.0 # 0.009482037 # Volume fraction
inCO2_ = 0.0 # 0.0003864118 # Volume fraction
inCO_  = 0 
inS_   = 0

In [41]:
#  ==========================
#  = Parse chemical formula =
#  ==========================

# Standardwerte für Atome
elements = {'C': 0, 'H': 0, 'O': 0, 'N': 0, 'Cl': 0}

# Suche nach Element-Symbol + optionaler Zahl
matches = re.findall(r'(Cl|C|H|O|N)(\d*\.?\d*)', fuel_formula, re.IGNORECASE)

for elem, count in matches:
    count = float(count) if count else 1
    elem = elem.capitalize()
    if elem in elements:
        elements[elem] += count

# Convert all atom numbers to floats
C = float(elements['C'])
H = float(elements['H'])
O = float(elements['O'])
N = float(elements['N'])
Cl = float(elements['Cl'])

# Throw error if formula does not include C or H
if ((C == 0) or (H == 0)):
    print("C and H must be included in the fuel")
    sys.exit()

In [42]:
print(C,H,O,N,Cl)

2.0 3.0 0.0 0.0 1.0


## Weights (g/mol)

In [43]:
# component input
inX_ = C # Carbon
inY_ = H # Hydrogen
inZ_ = O # Oxygen
inV_ = N # Nitrogen
inW_ = Cl # Chlorine

In [44]:
# Molecular weights of C, H, O, N, Cl in g/mol
# base: MAKE_PERIODIC_TABLE in data.f90 - source: IUPAC weights
wC_  = 12.01070 # Carbon
wH_  =  1.00794 # Hydrogen
wO_  = 15.99940 # Oxygen
wN_  = 14.00670 # Nitrogen
wCl_ = 35.45300 # Chlorine
wO2_ = 2 * wO_
wCO2_ = wC_ + 2 * wO_
wH2O_ = 2 * wH_ + wO_
wCO_  = wC_ + wO_
wS_   = inXH_ * wH_ + (1 - inXH_) * wC_
wN2_  = 2 * wN_
wHCN_ = wH_ + wC_ + wN_ 
wHCl_ = wH_ + wCl_
# molecular weight of fuel
wF_   = inX_ * wC_ + inY_ * wH_ + inZ_ * wO_ + inV_ * wN_ + inW_ * wCl_

In [45]:
print(inX_ , wC_ , inY_ , wH_ , inZ_ , wO_ , inV_ , wN_ , inW_ , wCl_)

2.0 12.0107 3.0 1.00794 0.0 15.9994 0.0 14.0067 1.0 35.453


In [46]:
print(f'C:\t {wC_} g/mol')
print(f'H:\t {wH_} g/mol')
print(f'O:\t {wO_} g/mol\tO_2:\t{wO2_} g/mol')
print(f'N:\t {wN_} g/mol\tN_2:\t{wN2_} g/mol')
print(f'Cl:\t {wCl_} g/mol')
print('--')
print(f'CO:\t {wCO_} g/mol')
print(f'CO2:\t {wCO2_} g/mol')
print(f'H2O:\t {wH2O_} g/mol')
print(f'HCN:\t {wHCN_} g/mol')
print(f'HCl:\t {wHCl_} g/mol')
print(f'Soot:\t {wS_} g/mol')
print('--')
print(f'wF ({fuel_name},{fuel_formula}):\t{wF_} g/mol')

C:	 12.0107 g/mol
H:	 1.00794 g/mol
O:	 15.9994 g/mol	O_2:	31.9988 g/mol
N:	 14.0067 g/mol	N_2:	28.0134 g/mol
Cl:	 35.453 g/mol
--
CO:	 28.0101 g/mol
CO2:	 44.0095 g/mol
H2O:	 18.01528 g/mol
HCN:	 27.02534 g/mol
HCl:	 36.46094 g/mol
Soot:	 12.0107 g/mol
--
wF (PVC,C2H3Cl):	62.49822 g/mol


In [47]:
# Air Composition
## Vol Norm
airVolO2_ = inO2_ / (inO2_ + inN2_ + inH2O_ + inCO2_)
airVolN2_ = inN2_ / (inO2_ + inN2_ + inH2O_ + inCO2_)
airVolH2O_ = inH2O_ / (inO2_ + inN2_ + inH2O_ + inCO2_)
airVolCO2_ = inCO2_ / (inO2_ + inN2_ + inH2O_ + inCO2_)
airVolNorm_check = airVolO2_ + airVolN2_ + airVolH2O_ + airVolCO2_

airMassO2_  = airVolO2_ * wO2_ / (airVolO2_ * wO2_ + airVolN2_ * wN2_ + airVolH2O_ * wH2O_ + airVolCO2_ * wCO2_)
airMassN2_  = airVolN2_ * wN2_ / (airVolO2_ * wO2_ + airVolN2_ * wN2_ + airVolH2O_ * wH2O_ + airVolCO2_ * wCO2_)
airMassH2O_ = airVolH2O_ * wH2O_ / (airVolO2_ * wO2_ + airVolN2_ * wN2_ + airVolH2O_ * wH2O_ + airVolCO2_ * wCO2_)
airMassCO2_ = airVolCO2_ * wCO2_ / (airVolO2_ * wO2_ + airVolN2_ * wN2_ + airVolH2O_ * wH2O_ + airVolCO2_ * wCO2_)
airMass_check = airMassO2_ + airMassN2_ + airMassH2O_ + airMassCO2_

## Vol Fraction
print('Species\tVol Fraction\tVol Norm\tMass Fraction')
print(f'O2\t{inO2_}\t{airVolO2_}\t{airMassO2_}')
print(f'N2\t{inN2_}\t{airVolN2_}\t{airMassN2_}')
print(f'H2O\t{inH2O_}\t{airVolH2O_}\t{airMassH2O_}')
print(f'CO2\t{inCO2_}\t{airVolCO2_}\t{airMassCO2_}')
print('------------')
print(f'check\t\t{airVolNorm_check}\t{airMass_check}')

Species	Vol Fraction	Vol Norm	Mass Fraction
O2	0.2075588	0.20962750907328975	0.23251633460205018
N2	0.7825727	0.7903724909267102	0.7674836653979498
H2O	0.0	0.0	0.0
CO2	0.0	0.0	0.0
------------
check		1.0	1.0


In [48]:
# Stoichiometric Coeff
nuS_ = wF_ / wS_ * inYS_

nuCO_ = wF_ / wCO_ * inYCO_

nuHCN_ = wF_ / wHCN_ * inYHCN_

In [49]:
# HRR input
release_ = 13100 # Release per Unit O2 - EPUMO2
hrr_ = 500 # Peak HRR wanted in kW

In [50]:
matrix = np.array([[1,0,0,0,0],
                  [0,2,0,0,1],
                  [2,1,0,-2,0],
                  [0,0,2,0,0],
                  [0,0,0,0,1]], dtype=float)

In [51]:
RHS = np.array([[inX_ - nuCO_ - nuS_ * (1 - inXH_) - nuHCN_],
                [inY_ - nuS_ * inXH_ - nuHCN_],
                [inZ_ - nuCO_],
                [inV_ - nuHCN_],
                [inW_]], dtype=float)

In [52]:
inverse_matrix = np.linalg.inv(matrix)

In [53]:
solution = np.matmul(inverse_matrix, RHS)

In [54]:
solution

array([[0.00143266],
       [1.        ],
       [0.        ],
       [0.88409623],
       [1.        ]])

In [55]:
nuCO2_ = solution[0].item()
nuH2O_ = solution[1].item()
nuN2_  = solution[2].item()
nuO2_  = solution[3].item()
nuHCl_ = solution[4].item()

In [56]:
# balances chemical equation
# C1 * Fuel + C2 * Air -> C3 * Products

_c1_ = 1 # Fuel
_c2_ = nuO2_ / airVolO2_

In [57]:
# Product Coeff with Air
prodHCl_  = nuHCl_
prodCO2_  = nuCO2_ + _c2_ * airVolCO2_
prodH2O_  = nuH2O_ + _c2_ * airVolH2O_
prodCO_   = nuCO_
prodSOOT_ = nuS_
prodHCN_  = nuHCN_
prodN2_   = nuN2_ + _c2_ * airVolN2_

sumProd_ = prodHCl_ + prodCO2_ + prodH2O_ + prodCO_ + prodSOOT_ + prodHCN_ + prodN2_

prodHClnorm_  = prodHCl_ / sumProd_
prodCO2norm_  = prodCO2_ / sumProd_
prodH2Onorm_  = prodH2O_ / sumProd_
prodCOnorm_   = prodCO_ / sumProd_
prodSOOTnorm_ = prodSOOT_ / sumProd_
prodHCNnorm_  = prodHCN_ / sumProd_
prodN2norm_   = prodN2_ / sumProd_

In [58]:
# balanced chemical equation
# C1 * Fuel + C2 * Air -> C3 * Products
_c3_ = sumProd_
print('{0} * Fuel + {1} * Air -> {2} * Products'.format(_c1_, _c2_, _c3_))

1 * Fuel + 4.217462835896426 * Air -> 7.333366606998286 * Products


In [59]:
# balanced chemical equation
## Fuel
C1 = 1
C1_C = inW_
C1_H = inX_
C1_O = inY_
C1_N = inV_
C1_Cl = inW_

## Air
C2     = nuO2_ / airVolO2_
C2_O2  = airVolO2_
C2_N2  = airVolN2_
C2_H2O = airVolH2O_
C2_CO2 = airVolCO2_

## Product
C3      = sumProd_
C3_HCl  = prodHClnorm_
C3_CO2  = prodCO2norm_
C3_H2O  = prodH2Onorm_
C3_CO   = prodCOnorm_ 
C3_SOOT = prodSOOTnorm_
C3_HCN  = prodHCNnorm_ 
C3_N2   = prodN2norm_

In [60]:
print('Air = {0} * ( {1}*O2 + {2}*N2 + {3}*H2O + {4}*CO2)'.format(C2, C2_O2, C2_N2, C2_H2O, C2_CO2))

Air = 4.217462835896426 * ( 0.20962750907328975*O2 + 0.7903724909267102*N2 + 0.0*H2O + 0.0*CO2)


In [61]:
print('Products = {0} * ({1}*HCl + {2}*CO2 + {3}*H2O + {4}*CO + {5}*SOOT + {6}*HCN + {7}*N2)'.format(
    C3, C3_HCl, C3_CO2, C3_H2O, C3_CO, C3_SOOT, C3_HCN, C3_N2))

Products = 7.333366606998286 * (0.13636301764126896*HCl + 0.00019536233350097147*CO2 + 0.13636301764126896*H2O + 0.10436231700736208*CO + 0.1681683559416749*SOOT + 0.0*HCN + 0.4545479294349241*N2)


In [62]:
# calculated stoichiometric coefficients
print('Calculated Stoichiometric Coefficients')
print('\t nu_{O_2} \t', nuO2_)
print('\t nu_{CO_2} \t', nuCO2_)
print('\t nu_{H2O} \t', nuH2O_)
print('\t nu_{CO} \t', nuCO_)
print('\t nu_{SOOT} \t', nuS_)
print('\t nu_{N} \t', nuN2_)
print('\t nu_{HCN} \t', nuHCN_)
print('\t nu_{HCl} \t', nuHCl_)

Calculated Stoichiometric Coefficients
	 nu_{O_2} 	 0.8840962288981404
	 nu_{CO_2} 	 0.0014326636127612868
	 nu_{H2O} 	 1.0
	 nu_{CO} 	 0.7653271305707584
	 nu_{SOOT} 	 1.2332402058164804
	 nu_{N} 	 0.0
	 nu_{HCN} 	 0.0
	 nu_{HCl} 	 1.0


In [63]:
# Yields (mass product / mass fuel) - FDS equation, No Air)
yO2_  = nuO2_ * wO2_ / wF_
yHCl_ = nuHCl_ * wHCl_ / wF_
yCO2_ = nuCO2_ * wCO2_ / wF_
yH2O_ = nuH2O_ * wH2O_ / wF_
yCO_  = nuCO_ * wCO_ / wF_
yS_   = nuS_ * wS_ / wF_
yHCN_ = nuHCN_ * wHCN_ / wF_
yN2_  = nuN2_ * wN2_ / wF_

In [64]:
# calculated stoichiometric coefficients
print('Yields (mass product / mass fuel) - FDS equation, No Air')
print('\t Y_{O_2} \t', yO2_)
print('\t Y_{HCl} \t', yHCl_)
print('\t Y_{CO_2} \t', yCO2_)
print('\t Y_{H2O} \t', yH2O_)
print('\t Y_{CO} \t', yCO_)
print('\t Y_{SOOT} \t', yS_)
print('\t Y_{HCN} \t', yHCN_)
print('\t Y_{N2} \t', yN2_)

Yields (mass product / mass fuel) - FDS equation, No Air
	 Y_{O_2} 	 0.4526531861109935
	 Y_{HCl} 	 0.5833916549943342
	 Y_{CO_2} 	 0.0010088416800641338
	 Y_{H2O} 	 0.28825268943659516
	 Y_{CO} 	 0.34299999999999997
	 Y_{SOOT} 	 0.237
	 Y_{HCN} 	 0.0
	 Y_{N2} 	 0.0


In [65]:
# Calculated Fuel Parameters
hoc_ = release_ * yO2_ # Heat of combustion in kJ/kg
m_ = hrr_ / hoc_ # flow at Peak HRR in kg/s

print('HoC:\t',hoc_)
print('dot_m:\t',m_) 

HoC:	 5929.756738054015
dot_m:	 0.08432049105678599


In [66]:
# combustion process for 1 kg fuel

FuelAir_ = 0 # kg
N2Air_ = airMassN2_ * yO2_ / airMassO2_
O2Air_ = airMassO2_ * yO2_ / airMassO2_
CO2Air_ = airMassCO2_ * yO2_ / airMassO2_
COAir_ = 0
H2OAir_ = airMassH2O_ * yO2_ / airMassO2_
SOOTAir_ = 0
HCNAir_ = 0
HClAir_ = 0

mass_air_ = FuelAir_ + N2Air_ + O2Air_ + CO2Air_ + COAir_ + H2OAir_ + SOOTAir_ + HCNAir_ + HClAir_

FuelFuel_ = 1
N2Fuel_ = 0
O2Fuel_ = 0
CO2Fuel_ = 0
COFuel_ = 0
H2OFuel_ = 0
SOOTFuel_ = 0
HCNFuel_ = 0
HClFuel_ = 0

mass_fuel_ = FuelFuel_ + N2Fuel_ + O2Fuel_ + CO2Fuel_ + COFuel_ + H2OFuel_ + SOOTFuel_ + HCNFuel_ + HClFuel_

FuelProd_ = 0
N2Prod_   = yN2_ + N2Air_
O2Prod_   = 0 
CO2Prod_  = yCO2_ + CO2Air_ 
COProd_   = yCO_
H2OProd_  = yH2O_ + H2OAir_
SOOTProd_ = yS_
HCNProd_  = yHCN_
HClProd_  = yHCl_

mass_product_ = FuelProd_ + N2Prod_ + O2Prod_ + CO2Prod_ + COProd_ + H2OProd_ + SOOTProd_ + HCNProd_ + HClProd_


FuelAirMassFrac_ = FuelAir_ / mass_air_
N2AirMassFrac_   = N2Air_ / mass_air_
O2AirMassFrac_   = O2Air_ / mass_air_
CO2AirMassFrac_  = CO2Air_ / mass_air_
COAirMassFrac_   = COAir_ / mass_air_
H2OAirMassFrac_  = H2OAir_ / mass_air_
SOOTAirMassFrac_ = SOOTAir_ / mass_air_
HCNAirMassFrac_  = HCNAir_ / mass_air_
HClAirMassFrac_  = HClAir_ / mass_air_

FuelFuelMassFrac_ = FuelFuel_ / mass_fuel_
N2FuelMassFrac_   = N2Fuel_ / mass_fuel_
O2FuelMassFrac_   = O2Fuel_ / mass_fuel_
CO2FuelMassFrac_  = CO2Fuel_ / mass_fuel_
COFuelMassFrac_   = COFuel_ / mass_fuel_
H2OFuelMassFrac_  = H2OFuel_ / mass_fuel_
SOOTFuelMassFrac_ = SOOTFuel_ / mass_fuel_
HCNFuelMassFrac_  = HCNFuel_ / mass_fuel_
HClFuelMassFrac_  = HClFuel_ / mass_fuel_

FuelProdMassFrac_ = FuelProd_ / mass_product_
N2ProdMassFrac_   = N2Prod_ / mass_product_
O2ProdMassFrac_   = O2Prod_ / mass_product_
CO2ProdMassFrac_  = CO2Prod_ / mass_product_
COProdMassFrac_   = COProd_ / mass_product_
H2OProdMassFrac_  = H2OProd_ / mass_product_
SOOTProdMassFrac_ = SOOTProd_ / mass_product_
HCNProdMassFrac_  = HCNProd_ / mass_product_
HClProdMassFrac_  = HClProd_ / mass_product_

In [67]:
# Volume Fraction of Products
FuelProdVolFrac_  = FuelProdMassFrac_ / wF_
N2ProdVolFrac_   = N2ProdMassFrac_ / wN2_
O2ProdVolFrac_   = O2ProdMassFrac_ / wO2_
CO2ProdVolFrac_  = CO2ProdMassFrac_ / wCO2_
COProdVolFrac_   = COProdMassFrac_ / wCO_
H2OProdVolFrac_  = H2OProdMassFrac_ / wH2O_
SOOTProdVolFrac_ = SOOTProdMassFrac_ / wS_
HCNProdVolFrac_  = HCNProdMassFrac_ / wHCN_
HClProdVolFrac_  = HClProdMassFrac_ / wHCl_

In [68]:
# Volume Fraction of Products
print('Mass Fraction of Products')
print('Fuel\t', FuelProdMassFrac_) 
print('N2\t', N2ProdMassFrac_)
print('O2\t', O2ProdMassFrac_)
print('CO2\t', CO2ProdMassFrac_)
print('CO\t', COProdMassFrac_)
print('H2O\t', H2OProdMassFrac_)
print('SOOT\t',SOOTProdMassFrac_)
print('HCN\t',HCNProdMassFrac_)
print('HCl\t', HClProdMassFrac_)

Mass Fraction of Products
Fuel	 0.0
N2	 0.5070335383117868
O2	 0.0
CO2	 0.0003423563987466511
CO	 0.11639908133319413
H2O	 0.0978202572660122
SOOT	 0.08042735357424784
HCN	 0.0
HCl	 0.19797741311601236


In [69]:
# Volume Fraction of Products
print('Volume Fraction of Products')
print('Fuel\t', FuelProdVolFrac_) 
print('N2\t', N2ProdVolFrac_)
print('O2\t', O2ProdVolFrac_)
print('CO2\t', CO2ProdVolFrac_)
print('CO\t', COProdVolFrac_)
print('H2O\t', H2OProdVolFrac_)
print('SOOT\t',SOOTProdVolFrac_)
print('HCN\t',HCNProdVolFrac_)
print('HCl\t', HClProdVolFrac_)

Volume Fraction of Products
Fuel	 0.0
N2	 0.018099678664917032
O2	 0.0
CO2	 7.779147655543714e-06
CO	 0.0041556110593391
H2O	 0.005429849398178224
SOOT	 0.006696308589361806
HCN	 0.0
HCl	 0.005429849398178224


---

In [70]:
print("&SPEC ID = '{0}', FORMULA = '{1}'/".format(fuel_name, fuel_formula))

&SPEC ID = 'PVC', FORMULA = 'C2H3Cl'/


In [71]:
print("&SPEC ID = 'OXYGEN',\tLUMPED_COMPONENT_ONLY = T /")
print("&SPEC ID = 'NITROGEN',\tLUMPED_COMPONENT_ONLY = T /")
print("&SPEC ID = 'HYDROGEN CHLORIDE',\tLUMPED_COMPONENT_ONLY = T /")
print("&SPEC ID = 'WATER VAPOR',\tLUMPED_COMPONENT_ONLY = T /")
print("&SPEC ID = 'CARBON MONOXIDE',\tLUMPED_COMPONENT_ONLY = T /")
print("&SPEC ID = 'CARBON DIOXIDE',\tLUMPED_COMPONENT_ONLY = T /")
print("&SPEC ID ='SOOT', MASS_EXTINCTION_COEFFICIENT=8700., LUMPED_COMPONENT_ONLY= T /")

&SPEC ID = 'OXYGEN',	LUMPED_COMPONENT_ONLY = T /
&SPEC ID = 'NITROGEN',	LUMPED_COMPONENT_ONLY = T /
&SPEC ID = 'HYDROGEN CHLORIDE',	LUMPED_COMPONENT_ONLY = T /
&SPEC ID = 'WATER VAPOR',	LUMPED_COMPONENT_ONLY = T /
&SPEC ID = 'CARBON MONOXIDE',	LUMPED_COMPONENT_ONLY = T /
&SPEC ID = 'CARBON DIOXIDE',	LUMPED_COMPONENT_ONLY = T /
&SPEC ID ='SOOT', MASS_EXTINCTION_COEFFICIENT=8700., LUMPED_COMPONENT_ONLY= T /


In [72]:
print("&SPEC\tID = 'AIR', BACKGROUND = T,")
print("\tSPEC_ID(1)='OXYGEN',\tVOLUME_FRACTION(1)={0:.8f},".format(airVolO2_))
print("\tSPEC_ID(2)='NITROGEN',\tVOLUME_FRACTION(2)={0:.8f},".format(airVolN2_))
print("\tSPEC_ID(3)='WATER VAPOR',\tVOLUME_FRACTION(3)={0:.8f},".format(airVolH2O_))
print("\tSPEC_ID(4)='CARBON DIOXIDE',\tVOLUME_FRACTION(4)={0:.8f}".format(airVolCO2_))
print("\t/")

&SPEC	ID = 'AIR', BACKGROUND = T,
	SPEC_ID(1)='OXYGEN',	VOLUME_FRACTION(1)=0.20962751,
	SPEC_ID(2)='NITROGEN',	VOLUME_FRACTION(2)=0.79037249,
	SPEC_ID(3)='WATER VAPOR',	VOLUME_FRACTION(3)=0.00000000,
	SPEC_ID(4)='CARBON DIOXIDE',	VOLUME_FRACTION(4)=0.00000000
	/


In [73]:
print("&SPEC\tID = 'PRODUCTS',")
print("\tSPEC_ID(1)='HYDROGEN CHLORIDE',\tVOLUME_FRACTION(1)={0:.6f},".format(C3_HCl))
print("\tSPEC_ID(2)='WATER VAPOR',\tVOLUME_FRACTION(2)={0:.6f},".format(C3_H2O))
print("\tSPEC_ID(3)='CARBON MONOXIDE',\tVOLUME_FRACTION(3)={0:.6f},".format(C3_CO))
print("\tSPEC_ID(4)='CARBON DIOXIDE',\tVOLUME_FRACTION(4)={0:.6f},".format(C3_CO2))
print("\tSPEC_ID(5)='HYDROGEN CYANIDE',\tVOLUME_FRACTION(5)={0:.6f},".format(C3_HCN))
print("\tSPEC_ID(6)='NITROGEN',\tVOLUME_FRACTION(6)={0:.6f}".format(C3_N2))
print("\tSPEC_ID(7)='SOOT',\tVOLUME_FRACTION(7)={0:.6f}".format(C3_SOOT))
print("\t/")

&SPEC	ID = 'PRODUCTS',
	SPEC_ID(1)='HYDROGEN CHLORIDE',	VOLUME_FRACTION(1)=0.136363,
	SPEC_ID(2)='WATER VAPOR',	VOLUME_FRACTION(2)=0.136363,
	SPEC_ID(3)='CARBON MONOXIDE',	VOLUME_FRACTION(3)=0.104362,
	SPEC_ID(4)='CARBON DIOXIDE',	VOLUME_FRACTION(4)=0.000195,
	SPEC_ID(5)='HYDROGEN CYANIDE',	VOLUME_FRACTION(5)=0.000000,
	SPEC_ID(6)='NITROGEN',	VOLUME_FRACTION(6)=0.454548
	SPEC_ID(7)='SOOT',	VOLUME_FRACTION(7)=0.168168
	/


In [74]:
print("&REAC FUEL='{0}', SPEC_ID_NU ='{0}','AIR','PRODUCTS',".format(fuel_name))
print("NU=-{0:.6f},-{1:.6f},{2:.6f}, HEAT_OF_COMBUSTION={3:.2f} /".format(_c1_,_c2_,_c3_,hoc_))

&REAC FUEL='PVC', SPEC_ID_NU ='PVC','AIR','PRODUCTS',
NU=-1.000000,-4.217463,7.333367, HEAT_OF_COMBUSTION=5929.76 /
